In [65]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras.models import Model
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import Adam


# Load data
df = pd.read_csv("jobs_in_data.csv")

# Split the 'salary_in_usd' column by 1000
df['salary_in_usd'] /= 10000

# Feature 1: Calculate the ratio of "salary_in_usd" to "salary"
df['salary_ratio'] = df['salary_in_usd'] / df['salary']

# Feature 2: Map experience levels to ordinal numbers
experience_mapping = {
    'Entry-level': 1,
    'Mid-level': 2,
    'Senior': 3,
    'Executive': 4
}
df['experience_level_encoded'] = df['experience_level'].map(experience_mapping)

# Feature 3: Calculate the percentile rank of each salary within its job category
df['Percentile'] = df.groupby('job_category')['salary'].rank(pct=True)

# Normalize the percentile ranks to a scale of 0 to 1
min_percentile = df['Percentile'].min()
max_percentile = df['Percentile'].max()
df['Normalized_Salary_within_Job_Category'] = (df['Percentile'] - min_percentile) / (max_percentile - min_percentile)

# Drop the temporary 'Percentile' column if you don't need it anymore
df.drop(columns=['Percentile'], inplace=True)

# Define features and target variable
X_numerical = df.select_dtypes(include=np.number).drop(
    columns=["salary_in_usd", "salary"])  # Select only numeric columns
y = df["salary_in_usd"]

# Scale numerical data
scaler = StandardScaler()
X_numerical_scaled = scaler.fit_transform(X_numerical)

# Split data into train and test sets
X_train_num, X_test_num, y_train, y_test = train_test_split(X_numerical_scaled, y, test_size=0.2, random_state=42)

# Reshape the input data to fit CNN input shape
X_train_reshaped = X_train_num.reshape(X_train_num.shape[0], X_train_num.shape[1], 1)
X_test_reshaped = X_test_num.reshape(X_test_num.shape[0], X_test_num.shape[1], 1)

# Define CNN model
input_layer = Input(shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]))
conv1 = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(input_layer)
pool1 = MaxPooling1D(pool_size=2)(conv1)
conv2 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(pool1)
pool2 = MaxPooling1D(pool_size=2)(conv2)
conv3 = Conv1D(filters=128, kernel_size=1, activation='relu', padding='valid')(pool2)
flatten = Flatten()(conv3)
dense1 = Dense(128, activation='relu')(flatten)
dropout = Dropout(0.5)(dense1)
output_layer = Dense(1, activation='linear')(dropout)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
optimizer = Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mse'])

# Train the model
history = model.fit(X_train_reshaped, y_train, epochs=100, batch_size=64, validation_split=0.1)

# Evaluate the model
y_pred = model.predict(X_test_reshaped)
mse = mean_squared_error(y_test, y_pred)
print("RMSE:",np.sqrt(mse))

Epoch 1/100
106/106 [==============================] - 2s 8ms/step - loss: 26.1646 - mse: 26.1646 - val_loss: 8.4619 - val_mse: 8.4619
Epoch 2/100
106/106 [==============================] - 1s 6ms/step - loss: 12.8970 - mse: 12.8970 - val_loss: 10.7619 - val_mse: 10.7619
Epoch 3/100
106/106 [==============================] - 1s 5ms/step - loss: 12.4062 - mse: 12.4062 - val_loss: 7.8137 - val_mse: 7.8137
Epoch 4/100
106/106 [==============================] - 1s 5ms/step - loss: 11.9907 - mse: 11.9907 - val_loss: 7.8607 - val_mse: 7.8607
Epoch 5/100
106/106 [==============================] - 0s 5ms/step - loss: 12.0113 - mse: 12.0113 - val_loss: 7.7849 - val_mse: 7.7849
Epoch 6/100
106/106 [==============================] - 1s 6ms/step - loss: 11.5079 - mse: 11.5079 - val_loss: 8.4796 - val_mse: 8.4796
Epoch 7/100
106/106 [==============================] - 1s 6ms/step - loss: 11.9898 - mse: 11.9898 - val_loss: 6.8119 - val_mse: 6.8119
Epoch 8/100
106/106 [==============================] 